In [39]:
import multiset

class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_lorentz_rank(self):
        return self.lorentz_rank
    def get_spinor_rank(self):
        return self.spinor_rank
    def get_spinor_rank_conj(self):
        return self.spinor_rank_conj
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq

In [3]:
D = field('D', 1, 1, 0, 0)
F = field('F', 2, 2, 0, 0)
S = field('Pb_S_P', 3, 0, 0, 0)
V = field('Pb_V_P', 3, 0, 0, 0)
T = field('Pb_T_P', 3, 0, 0, 0)
Vp = field('Pb_Vp_P', 3, 0, 0, 0)
Sp = field('Pb_Sp_P', 3, 0, 0, 0)

In [4]:
# field_combo indicates how many of each field are contained in the term in question
field_dict = {'D': 2, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
print(field_dict.keys())
print(field_dict.values())

dict_keys(['D', 'F', 'Pb_S_P', 'Pb_V_P', 'Pb_T_P', 'Pb_Vp_P', 'Pb_Sp_P'])
dict_values([2, 1, 0, 0, 0, 0, 0])


In [5]:
list(field_dict.values())

[2, 1, 0, 0, 0, 0, 0]

In [6]:
def lorentzRanks_list(field_dict):
    symbol_list = list(field_dict.keys())
    numFields_list = list(field_dict.values())
    N_fieldTypes = len(symbol_list)
    lorentzRanks = numFields_list
    # if field is 'F', 'T', number of lorentz indices is twice the number of fields
    for i in range(N_fieldTypes):
        if symbol_list[i] == 'F' or symbol_list[i] == 'Pb_T_P':
            lorentzRanks[i] *= 2
    return lorentzRanks

     
def generate_lorentz_contractions(lorentzRanks_list):
    # EXPLANATION: recursively generateS all ways of lorentz contracting fields. generate a set of sets of contractions. all sets of
    # contractions should have the same number of contractions. take first index in lorentzRanks_list and contract it with every field with non-zero lorentz rank. 
    # for each such contraction, update the lorentzRanks_list by decrementing the lorentz rank of each contracted
    # field by one for each contracted index. append this contraction to each of the sets of contractions output 
    # by generate_lorentz_contractions() acting on the decremented lorentzRanks_list. 
    
    # base case: if the total number of free Lorentz indices is 0 or 1, return empty list of contraction sets 
    if sum(lorentzRanks_list) < 2:
        contraction_list_list = [[]]
        return contraction_list_list
    
    # set to store different sets of contractions
    contraction_list_list = []
    # variable to store number of distinct fields
    N_fieldTypes = len(lorentzRanks_list)
    
    # find first field of non-zero rank for first index of contraction
    for i in range(N_fieldTypes):
        if lorentzRanks_list[i] >= 1:
            i_start = i
            #print('i_start: ' + str(i_start))
            lorentzRanks_list[i_start] -= 1 #decrement lorentzRanks_list[i_start] by one
            break
            
    # find all ways of contracting first non-zero lorentz rank field with other fields or itself 
    for i in range(i_start, N_fieldTypes):
        #print(i)
        if lorentzRanks_list[i] >= 1:
            contraction = (i_start, i)
            lorentzRanks_list_old = lorentzRanks_list.copy()
            lorentzRanks_list_old[i] -= 1
            #print('lorentzRanks_list_old: ' + str(lorentzRanks_list_old))
            contraction_list_list_old = generate_lorentz_contractions(lorentzRanks_list_old)
            #print('contraction_list_list_old: ' + str(contraction_list_list_old))
            
            for contraction_list_old in contraction_list_list_old:
                #print(contraction_list_old)
                contraction_list = contraction_list_old
                contraction_list.append(contraction)
                #print(contraction_list)
                contraction_list_list.append(contraction_list) 
                #print(contraction_list_list)
    return contraction_list_list
        
        
def generate_lorentz_contractions_from_dict(field_dict):
    lorentzRanks = lorentzRanks_list(field_dict)
    contraction_list_list = generate_lorentz_contractions(lorentzRanks)
    return contraction_list_list
    
     
    

In [18]:
def reduce_set_of_contraction_sets(field_dict):
    contraction_list_list = generate_lorentz_contractions_from_dict(field_dict)
    contraction_set_set = set()
    for contraction_list in contraction_list_list:
    #print('contraction_list:' +str(contraction_list))
        contraction_set = FrozenMultiset(contraction_list)
        contraction_set_set.add(contraction_set)
    return contraction_set_set

# Test 1

In [7]:
lorentzRanks = lorentzRanks_list(field_dict)
lorentzRanks

[2, 2, 0, 0, 0, 0, 0]

In [8]:
generate_lorentz_contractions(lorentzRanks)

[[(1, 1), (0, 0)], [(0, 1), (0, 1)]]

In [9]:
field_dict2 = {'D': 2, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 2, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

generate_lorentz_contractions_from_dict(field_dict2)

[[(3, 3), (1, 1), (0, 0)],
 [(1, 3), (1, 3), (0, 0)],
 [(3, 3), (0, 1), (0, 1)],
 [(1, 3), (0, 3), (0, 1)],
 [(1, 3), (0, 1), (0, 3)],
 [(1, 1), (0, 3), (0, 3)]]

BUG: 4th and 5th contractions are redundant.

In [10]:
from multiset import FrozenMultiset
contraction_list_list = generate_lorentz_contractions_from_dict(field_dict2)
#print(contraction_list_list)
contraction_set_set = set()
for contraction_list in contraction_list_list:
    #print('contraction_list:' +str(contraction_list))
    contraction_set = FrozenMultiset(contraction_list)
    print('contraction_set:' +str(contraction_set))
    contraction_set_set.add(contraction_set)
    
contraction_set_set    
    

contraction_set:{(3, 3), (1, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 0)}
contraction_set:{(3, 3), (0, 1), (0, 1)}
contraction_set:{(1, 3), (0, 3), (0, 1)}
contraction_set:{(1, 3), (0, 1), (0, 3)}
contraction_set:{(1, 1), (0, 3), (0, 3)}


{FrozenMultiset({(1, 3): 2, (0, 0): 1}),
 FrozenMultiset({(1, 3): 1, (0, 3): 1, (0, 1): 1}),
 FrozenMultiset({(3, 3): 1, (1, 1): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (0, 1): 2}),
 FrozenMultiset({(1, 1): 1, (0, 3): 2})}

Removed redundant contraction set, but is there a way to avoid producing it in the first place?

In [11]:
field_dict4 = {'D': 4, 'F': 2, 'Pb_S_P': 0, 'Pb_V_P': 3, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

len(generate_lorentz_contractions_from_dict(field_dict4))

59

In [12]:
contraction_list_list4 = generate_lorentz_contractions_from_dict(field_dict4)
contraction_set_set4 = set()
for contraction_list in contraction_list_list4:
    #print('contraction_list:' +str(contraction_list))
    contraction_set = FrozenMultiset(contraction_list)
    print('contraction_set:' +str(contraction_set))
    contraction_set_set4.add(contraction_set)
    
#len(contraction_set_set4)
print(len(contraction_list_list4))
print(len(contraction_set_set4))

contraction_set:{(3, 3), (1, 1), (1, 1), (0, 0), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 0), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 0), (0, 0)}
contraction_set:{(1, 1), (1, 3), (1, 3), (0, 0), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 3), (0, 0), (0, 0)}
contraction_set:{(3, 3), (1, 1), (0, 1), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 1), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 1), (0, 3), (0, 1), (0, 0)}
contraction_set:{(1, 1), (1, 3), (0, 3), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 3), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 1), (0, 1), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 3), (0, 1), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 1), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 1), (0, 3), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 1), (0, 3), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 3), (0, 3), (0, 3), (0, 0)}
contraction_set:{(3, 3), (1, 1), (0, 1), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3)

# Test 2

In [13]:
field_dict3 = {'D': 0, 'F': 3, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

generate_lorentz_contractions_from_dict(field_dict3)

[[(1, 1), (1, 1), (1, 1)]]

BUG: can't distinguish between contractions on the same F and contractions on different F's.

# Test 3

In [15]:
field_dict5 = {'D': 3, 'F': 2, 'Pb_S_P': 0, 'Pb_V_P': 3, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

generate_lorentz_contractions_from_dict(field_dict3)

[[(3, 3), (1, 3), (1, 1), (0, 1), (0, 0)],
 [(3, 3), (1, 1), (1, 3), (0, 1), (0, 0)],
 [(1, 3), (1, 3), (1, 3), (0, 1), (0, 0)],
 [(3, 3), (1, 1), (1, 1), (0, 3), (0, 0)],
 [(1, 3), (1, 3), (1, 1), (0, 3), (0, 0)],
 [(1, 3), (1, 1), (1, 3), (0, 3), (0, 0)],
 [(1, 1), (1, 3), (1, 3), (0, 3), (0, 0)],
 [(3, 3), (1, 3), (1, 1), (0, 0), (0, 1)],
 [(3, 3), (1, 1), (1, 3), (0, 0), (0, 1)],
 [(1, 3), (1, 3), (1, 3), (0, 0), (0, 1)],
 [(3, 3), (1, 3), (0, 1), (0, 1), (0, 1)],
 [(3, 3), (1, 1), (0, 3), (0, 1), (0, 1)],
 [(1, 3), (1, 3), (0, 3), (0, 1), (0, 1)],
 [(3, 3), (1, 1), (0, 1), (0, 3), (0, 1)],
 [(1, 3), (1, 3), (0, 1), (0, 3), (0, 1)],
 [(1, 3), (1, 1), (0, 3), (0, 3), (0, 1)],
 [(1, 1), (1, 3), (0, 3), (0, 3), (0, 1)],
 [(3, 3), (1, 1), (1, 1), (0, 0), (0, 3)],
 [(1, 3), (1, 3), (1, 1), (0, 0), (0, 3)],
 [(1, 3), (1, 1), (1, 3), (0, 0), (0, 3)],
 [(1, 1), (1, 3), (1, 3), (0, 0), (0, 3)],
 [(3, 3), (1, 1), (0, 1), (0, 1), (0, 3)],
 [(1, 3), (1, 3), (0, 1), (0, 1), (0, 3)],
 [(1, 3), (

In [17]:
contraction_list_list5 = generate_lorentz_contractions_from_dict(field_dict5)
contraction_set_set5 = set()
for contraction_list in contraction_list_list5:
    #print('contraction_list:' +str(contraction_list))
    contraction_set = FrozenMultiset(contraction_list)
    print('contraction_set:' +str(contraction_set))
    contraction_set_set5.add(contraction_set)
    
#len(contraction_set_set4)
print(len(contraction_list_list5))
print(len(contraction_set_set5))
contraction_set_set5

contraction_set:{(3, 3), (1, 3), (1, 1), (0, 1), (0, 0)}
contraction_set:{(3, 3), (1, 1), (1, 3), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 3), (0, 1), (0, 0)}
contraction_set:{(3, 3), (1, 1), (1, 1), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 3), (1, 3), (0, 3), (0, 0)}
contraction_set:{(3, 3), (1, 3), (1, 1), (0, 0), (0, 1)}
contraction_set:{(3, 3), (1, 1), (1, 3), (0, 0), (0, 1)}
contraction_set:{(1, 3), (1, 3), (1, 3), (0, 0), (0, 1)}
contraction_set:{(3, 3), (1, 3), (0, 1), (0, 1), (0, 1)}
contraction_set:{(3, 3), (1, 1), (0, 3), (0, 1), (0, 1)}
contraction_set:{(1, 3), (1, 3), (0, 3), (0, 1), (0, 1)}
contraction_set:{(3, 3), (1, 1), (0, 1), (0, 1), (0, 3)}
contraction_set:{(1, 3), (1, 3), (0, 1), (0, 1), (0, 3)}
contraction_set:{(1, 3), (1, 1), (0, 3), (0, 3), (0, 1)}
contraction_set:{(1, 1), (1, 3), (0, 3), (0, 3), (0, 1)}
contraction_set:{(3, 3), (1, 1)

{FrozenMultiset({(1, 1): 2, (0, 3): 3}),
 FrozenMultiset({(3, 3): 1, (1, 1): 2, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (1, 1): 1, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (1, 1): 1, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(1, 3): 3, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 1): 1, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (0, 1): 3}),
 FrozenMultiset({(1, 3): 1, (1, 1): 1, (0, 3): 2, (0, 1): 1})}

In [19]:
# test reduce_set_of_contraction_sets()
reduce_set_of_contraction_sets(field_dict5)

{FrozenMultiset({(1, 1): 2, (0, 3): 3}),
 FrozenMultiset({(3, 3): 1, (1, 1): 2, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (1, 1): 1, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (1, 1): 1, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(1, 3): 3, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 1): 1, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (0, 1): 3}),
 FrozenMultiset({(1, 3): 1, (1, 1): 1, (0, 3): 2, (0, 1): 1})}

# Generate all Field Combinations up to a Given Mass Dimension

In [58]:
'''
def generate_field_combos(massDim):
    # EXPLANATION: generate terms recursively. iterate through fields list, if mass dimension of field is less than
    # massDim, append it to combo, 
    
    # base case
    if massDim < 1: 
        return [[]]
    
    # declare field types
    D = field('D', 1, 1, 0, 0)
    F = field('F', 2, 2, 0, 0)
    S = field('Pb_S_P', 3, 0, 0, 0)
    V = field('Pb_V_P', 3, 0, 0, 0)
    T = field('Pb_T_P', 3, 0, 0, 0)
    Vp = field('Pb_Vp_P', 3, 0, 0, 0)
    Sp = field('Pb_Sp_P', 3, 0, 0, 0)
    
    # make list of field types
    fields = [D, F, S, V, T, Vp, Sp] 
    
    # empty list to store different field combos
    list_of_field_combos = []
    
    for i in range(len(fields)):
        field_current = fields[i]
        massDim_current = field_current.get_massDim() 
        massDimRes = massDim - massDim_current
        if massDimRes >= 0:
            print(massDimRes)
            list_of_field_combos_old = generate_field_combos(massDimRes)
            for field_combo_old in list_of_field_combos_old:
                field_combo = field_combo_old
                field_combo.append(field_current.get_symbol())
                list_of_field_combos.append(field_combo)

    return list_of_field_combos
'''
'''
from multiset import Multiset

def generate_field_combos(massDim):
    # EXPLANATION: generate terms recursively. iterate through fields list, if mass dimension of field is less than
    # massDim, append it to combo, 
    
    # base case
    if massDim < 1: 
        return [[]]
    
    # declare field types
    D = field('D', 1, 1, 0, 0)
    F = field('F', 2, 2, 0, 0)
    S = field('Pb_S_P', 3, 0, 0, 0)
    V = field('Pb_V_P', 3, 0, 0, 0)
    T = field('Pb_T_P', 3, 0, 0, 0)
    Vp = field('Pb_Vp_P', 3, 0, 0, 0)
    Sp = field('Pb_Sp_P', 3, 0, 0, 0)
    
    # make list of field types
    fields = [D, F, S, V, T, Vp, Sp] 
    
    # empty list to store different field combos
    list_of_field_combos = []
    
    for i in range(len(fields)):
        field_current = fields[i]
        massDim_current = field_current.get_massDim() 
        massDimRes = massDim - massDim_current
        if massDimRes >= 0:
            print(massDimRes)
            list_of_field_combos_old = generate_field_combos(massDimRes)
            for field_combo_old in list_of_field_combos_old:
                field_combo = field_combo_old
                field_combo.append(field_current.get_symbol())
                list_of_field_combos.append(field_combo)

    return list_of_field_combos
'''

"\nfrom multiset import Multiset\n\ndef generate_field_combos(massDim):\n    # EXPLANATION: generate terms recursively. iterate through fields list, if mass dimension of field is less than\n    # massDim, append it to combo, \n    \n    # base case\n    if massDim < 1: \n        return [[]]\n    \n    # declare field types\n    D = field('D', 1, 1, 0, 0)\n    F = field('F', 2, 2, 0, 0)\n    S = field('Pb_S_P', 3, 0, 0, 0)\n    V = field('Pb_V_P', 3, 0, 0, 0)\n    T = field('Pb_T_P', 3, 0, 0, 0)\n    Vp = field('Pb_Vp_P', 3, 0, 0, 0)\n    Sp = field('Pb_Sp_P', 3, 0, 0, 0)\n    \n    # make list of field types\n    fields = [D, F, S, V, T, Vp, Sp] \n    \n    # empty list to store different field combos\n    list_of_field_combos = []\n    \n    for i in range(len(fields)):\n        field_current = fields[i]\n        massDim_current = field_current.get_massDim() \n        massDimRes = massDim - massDim_current\n        if massDimRes >= 0:\n            print(massDimRes)\n            list_o

In [147]:
def symbols_to_fields(field_combo_symbols):
    field_combo = []
    for field_symbol in field_combo_symbols:
        if field_symbol == 'D':
            field_combo.append(D)
            continue
        if field_symbol == 'F':
            field_combo.append(F)
            continue
        if field_symbol == 'Pb_S_P':
            field_combo.append(S)
            continue
        if field_symbol == 'Pb_V_P':
            field_combo.append(V)
            continue
        if field_symbol == 'Pb_T_P':
            field_combo.append(T)
            continue
        if field_symbol == 'Pb_Vp_P':
            field_combo.append(Vp)
            continue
        if field_symbol == 'Pb_Sp_P':
            field_combo.append(Sp)
            continue
         
    return field_combo

def totalMassDim(field_combo_symbols):
    field_combo = symbols_to_fields(field_combo_symbols)
    m = 0
    for field in field_combo:
        m += field.get_massDim()
    return m

def generate_field_combos(massDim):
    # EXPLANATION: Generate all multisets of the symbols [D, F, S, V, T, Vp, Sp] up to massDim. store in a set of 
    # multisets called set_of_field_multisets. generate new multisets by adding each field to multisets in a set
    # of multisets called front. remove a multiset from front after attempting to add each field to it. add an
    # element to front only if after a field has been added, its mass dimension is still less than massDim. 
    
    # declare field types
    D = field('D', 1, 1, 0, 0)
    F = field('F', 2, 2, 0, 0)
    S = field('Pb_S_P', 3, 0, 0, 0)
    V = field('Pb_V_P', 3, 0, 0, 0)
    T = field('Pb_T_P', 3, 0, 0, 0)
    Vp = field('Pb_Vp_P', 3, 0, 0, 0)
    Sp = field('Pb_Sp_P', 3, 0, 0, 0)
    
    # make list of field types
    fields = [D, F, S, V, T, Vp, Sp] 
    
    # set of multisets
    set_of_field_multisets = set() #set of multisets of field strings
    front = set() #set of multisets of field strings, set of multisets to add to to make new multisets
    
    # initialize front
    for item in fields: 
        if item.get_massDim() <= massDim:
            front.add(FrozenMultiset([item.get_symbol()]))
            set_of_field_multisets.add(FrozenMultiset([item.get_symbol()]))

    # build set_of_multisets
    while front:
        #print('')
        #print('front: ' + str(front))
        front_new = front.copy()
        for field_multiset in front:
            #print('field_multiset: ' + str(field_multiset))
            for item in fields:
                #print('field item: ' + str(item.get_symbol()))
                field_multiset_new = field_multiset.combine(FrozenMultiset([item.get_symbol()]))
                #print('field_multiset_new: ' + str(field_multiset_new))
                if totalMassDim(field_multiset_new) < massDim:
                    set_of_field_multisets.add(field_multiset_new)
                    front_new.add(field_multiset_new)
                    #print('ADD TO FRONT: ' + str(field_multiset_new))
                if totalMassDim(field_multiset_new) == massDim:
                    set_of_field_multisets.add(field_multiset_new)
            front_new.remove(field_multiset)
        front = front_new
        #print('front_new: ' + str(front_new))
        #print('')
    
    return set_of_field_multisets
                    
                    
    
    
    
    
    

In [148]:
massDim = 5
for item in generate_field_combos(massDim):
    print(item)

{D, D, Pb_S_P}
{D, F, F}
{Pb_Vp_P, D}
{D, D, D, D, D}
{D, D, Pb_V_P}
{D, D}
{D, D, D}
{F, Pb_V_P}
{D, Pb_V_P}
{D, D, D, D}
{Pb_V_P}
{D}
{Pb_Vp_P, D, D}
{F, Pb_S_P}
{Pb_Vp_P, F}
{D, Pb_S_P}
{Pb_T_P, F}
{Pb_Sp_P}
{Pb_Vp_P}
{Pb_T_P, D}
{D, Pb_Sp_P}
{Pb_T_P}
{D, F}
{F, F}
{F, Pb_Sp_P}
{D, D, F}
{F}
{D, D, Pb_Sp_P}
{Pb_T_P, D, D}
{D, D, D, F}
{Pb_S_P}


Tested for massdim up to 5